In [33]:
# Random data generation
import pandas as pd
import numpy as np

# Create a sample dataset with multiple issues
np.random.seed(42)

data = {
    "Customer_ID": np.arange(1, 101),
    "Age": np.random.randint(18, 70, 100),
    "Region": np.random.choice(["North", "South", "East", "West"], 100),
    "Product": np.random.choice(["A", "B", "C"], 100),
    "Rating": np.random.uniform(1, 5, 100)
}

# Create DataFrame
df = pd.DataFrame(data)

# Introduce missing values
df.loc[::10, "Age"] = np.nan  # Missing 10% in Age column
df.loc[::15, "Region"] = np.nan  # Missing 6.67% in Region column

# Add some duplicate rows using pd.concat
df_duplicates = pd.concat([df, df.iloc[::5]], ignore_index=True)

# Introduce some outliers in the Rating column
outlier_indices = df_duplicates.iloc[::20].index  # Select every 20th row for outliers
df_duplicates.loc[outlier_indices, "Rating"] = np.random.uniform(10, 25, len(outlier_indices))  # Random outlier values

# Introduce some constant values in Product column
df_duplicates["Product"] = df_duplicates["Product"].replace({"A": "Special", "B": "Special"})

# Shuffle DataFrame
df_duplicates = df_duplicates.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the first few rows
df_duplicates.head()


,Customer_ID,Age,Region,Product,Rating
0,45,38.0,East,Special,3.580413
1,48,35.0,South,C,1.920741
2,5,60.0,North,C,3.233174
3,56,37.0,East,Special,4.533976
4,27,29.0,North,C,1.703701


# Guided Project: Data Preprocessing

In this guided project, we will use a synthetic dataset with multiple issues and walk through the process of cleaning the data. The following steps will be covered:

1. **Imputation**: Filling missing values.
2. **Row and Column Removal**: Removing rows or columns with issues.
3. **Encoding**: Applying encoding techniques to categorical data.

Let's begin!


## Step 1: Handling Missing Values

Missing values are a common issue in real-world datasets. 
Let's begin by identifying where these exist.


In [34]:
# Check for missing values
df.isnull().sum()

Customer_ID     0
Age            10
Region          7
Product         0
Rating          0
dtype: int64

In this dataset, we have missing values in the `Age` and `Region` columns.

We will use **median imputation** for the `Age` column and **mode imputation** for the `Region` column.

### Imputation Strategy:
- For **Age**: Use the median to fill in missing values.
- For **Region**: Use the mode (most frequent value) to fill in missing values.

Let's start by addressing the missing data.

In [35]:
# Impute missing values
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Region"].fillna(df["Region"].mode()[0], inplace=True)

# Display the result
df.isnull().sum()

C:\Users\W T\AppData\Local\Temp\ipykernel_3160\164178172.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)
C:\Users\W T\AppData\Local\Temp\ipykernel_3160\164178172.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

Customer_ID    0
Age            0
Region         0
Product        0
Rating         0
dtype: int64

## Step 2: Removing Duplicates and Unnecessary Columns

The dataset contains duplicate rows, which we will remove to ensure data integrity. Additionally, we will check if any columns are redundant or need to be removed.

### Actions:
1. Remove duplicate rows.
2. If any unnecessary columns exist, we will drop them.

Let's bgin by checking for duplicate rows.

In [36]:
# Check for and display duplicate rows in the dataset
duplicates_before = df_duplicates[df_duplicates.duplicated()]
duplicates_before

,Customer_ID,Age,Region,Product,Rating
21,46,26.0,NaN,Special,3.675696
27,71,NaN,North,Special,4.720067
47,91,NaN,NaN,Special,1.283764
51,36,32.0,West,C,3.035256
55,66,53.0,East,Special,2.073899
71,86,31.0,East,C,3.446055
74,56,37.0,East,Special,4.533976
77,51,NaN,North,Special,4.074216
88,6,25.0,West,Special,4.530545
95,26,50.0,East,Special,4.977830


In [37]:
# Remove duplicate rows
df_no_duplicates = df_duplicates.drop_duplicates()
# Check that duplicates have been removed
duplicates_after = df_no_duplicates[df_no_duplicates.duplicated()]
duplicates_after

,Customer_ID,Age,Region,Product,Rating


## Step 3: Handling Categorical Data (Encoding)

The dataset contains categorical columns, `Region` and `Product`, which we need to convert to numerical format.

We'll apply **One-Hot Encoding** for the `Region` column and **Label Encoding** for the `Product` column.

### Actions:
- **One-Hot Encoding**: Apply to the `Region` column.
- **Label Encoding**: Apply to the `Product` column.

Let's apply the encoding techniques.


In [17]:
# One-Hot Encoding for Region
df = pd.get_dummies(df, columns=["Region"], prefix="Region")

# Label Encoding for Product
df["Product_Encoded"] = df["Product"].map({"A": 0, "B": 1, "C": 2, "Special": 3})

# Display the result
df.head()


,Customer_ID,Age,Product,Rating,Region_East,Region_North,Region_South,Region_West,Product_Encoded
0,1,42.0,B,3.066544,True,False,False,False,1
1,2,69.0,B,2.043317,False,True,False,False,1
2,3,46.0,C,4.985015,False,True,False,False,2
3,4,32.0,B,4.861677,True,False,False,False,1
4,5,60.0,C,3.233174,False,True,False,False,2


## Final Cleaned Dataset

We have now cleaned the data by addressing missing values, removing duplicates, and encoding categorical columns. Below is the cleaned dataset ready for analysis or further steps.

### Next Steps:
- We could now move on to exploratory data analysis (EDA) or proceed with model training (in the next phase).


In [ ]:
# The final cleaned dataset
df.head()

,Customer_ID,Age,Product,Rating,Region_East,Region_North,Region_South,Region_West,Product_Encoded
0,1,42.0,B,3.066544,True,False,False,False,1
1,2,69.0,B,2.043317,False,True,False,False,1
2,3,46.0,C,4.985015,False,True,False,False,2
3,4,32.0,B,4.861677,True,False,False,False,1
4,5,60.0,C,3.233174,False,True,False,False,2
